In [1]:
import torch
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from glob import glob
import os
import pandas as pd
from torchvision import io
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

from PIL import Image, ImageFont, ImageDraw 

from components.dataset import SoilDataset, SoilDataset_phone
from components.mymodel import get_model

In [2]:
def save_result(img:Image.Image, label:str, predict:str, picname:str):
    draw = ImageDraw.Draw(img)
    font = ImageFont.load_default()
    # font.
    import cv2
    font_size = 30
    font_path = os.path.join(cv2.__path__[0],'qt','fonts','DejaVuSans.ttf')
    font = ImageFont.truetype(font_path, size=font_size)
    draw.text((10, 10),f"Label:{label}",(0,0,0), font=font)
    draw.text((10, 40 + font_size),f"Predict:{predict}",(0,0,0), font=font)
    img.save(picname)

In [5]:
def run_result(model_name:str, image_set:str):
    preprocess = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        # transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    model = get_model(model_name=model_name, image_set=image_set)
    dataset = SoilDataset_phone('./dataset/phones', image_set, transform=preprocess)
    loader = DataLoader(dataset=dataset, batch_size=1, shuffle=False, num_workers=1)
    # toPIL = transforms.transforms.ToPILImage()
    result_path = os.path.join('./result/',model_name,image_set)
    if(os.path.exists(result_path) == False):
        os.makedirs(result_path)

    pic_names = []
    ys = []
    yhats = []
    for X,y,pic_name in loader:
        yhat = model(X).detach()[0][0]
        img = Image.open(f'./dataset/phones/{image_set}/{pic_name[0]}')
        y = y[0]
        pic_name = os.path.join(result_path,pic_name[0])
        save_result(img, y.numpy(), yhat.numpy(), pic_name)

        _,pic_name = os.path.split(pic_name)
        pic_names.append(pic_name)
        ys.append(float(y))
        yhats.append(float(yhat))
        # break
    return pic_names, ys, yhats

In [6]:
image_set_list = next(os.walk('./dataset/phones/'))[1]
# mobilenet_v3_large
# resnet50
# efficientnet_v2_l
# alexnet
model_name = 'mobilenet_v3_large'
# for image_set in image_set_list:
pic_names, ys, yhats = run_result(model_name,'All')

Found 2355 images in ./dataset/phones/All.


KeyboardInterrupt: 

In [ ]:
import pandas as pd
df = pd.DataFrame()
df['pic_name'] = pic_names
df['OM'] = ys
df['predict'] = yhats

In [ ]:
df

,pic_name,OM,predict
0,./result/alexnet/All/(1)_A_0.27_65S-1_C_R_1.JPG,tensor(0.2700),tensor(0.3317)
1,./result/alexnet/All/(1)_A_0.27_65S-1_C_R_2.JPG,tensor(0.2700),tensor(0.4124)
2,./result/alexnet/All/(1)_A_0.27_65S-1_C_R_3.JPG,tensor(0.2700),tensor(0.4200)
3,./result/alexnet/All/(1)_A_0.27_65S-1_C_R_4.JPG,tensor(0.2700),tensor(0.3021)
4,./result/alexnet/All/(1)_A_0.27_65S-1_C_R_5.JPG,tensor(0.2700),tensor(0.3334)
...,...,...,...
2350,./result/alexnet/All/(99)_S_0.86_65S-197_M_R_1...,tensor(0.8600),tensor(0.5086)
2351,./result/alexnet/All/(99)_S_0.86_65S-197_M_R_2...,tensor(0.8600),tensor(0.6553)
2352,./result/alexnet/All/(99)_S_0.86_65S-197_M_R_3...,tensor(0.8600),tensor(0.6624)
2353,./result/alexnet/All/(99)_S_0.86_65S-197_M_R_4...,tensor(0.8600),tensor(0.6350)


In [ ]:
# pic_names_new = []
# for name in pic_names:
#     _,pic_name = os.path.split(name)
#     pic_names_new.append(pic_name)

# y_new = []
# for y in ys:
#     y_new.append(float(y))

# yhat_new = []
# for yhat in yhats:
#     yhat_new.append(float(yhat))

In [ ]:
# df_new = pd.DataFrame()
# df_new['pic_name'] = pic_names_new
# df_new['OM'] = y_new
# df_new['predict'] = yhat_new
# df_new

,pic_name,OM,predict
0,(1)_A_0.27_65S-1_C_R_1.JPG,0.27,0.331668
1,(1)_A_0.27_65S-1_C_R_2.JPG,0.27,0.412420
2,(1)_A_0.27_65S-1_C_R_3.JPG,0.27,0.420040
3,(1)_A_0.27_65S-1_C_R_4.JPG,0.27,0.302122
4,(1)_A_0.27_65S-1_C_R_5.JPG,0.27,0.333441
...,...,...,...
2350,(99)_S_0.86_65S-197_M_R_1.jpg,0.86,0.508597
2351,(99)_S_0.86_65S-197_M_R_2.jpg,0.86,0.655261
2352,(99)_S_0.86_65S-197_M_R_3.jpg,0.86,0.662441
2353,(99)_S_0.86_65S-197_M_R_4.jpg,0.86,0.635019


In [ ]:
df_new.to_csv('result.csv')